In [1]:
import os
import sys

sys.path.append(os.path.abspath("../../"))


import matplotlib.pyplot as plt
import my_utils as utils
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer

from utils.collator import SequenceGenerationCollator
from utils.model import MmLlamaConcat, MmLlamaConfig
from utils.processor import MmLlamaProcessor

BASE_PATH = os.path.join("results", "early_concat")
INSTRUCTERC_BASE_PATH = os.path.join("results", "instructerc")

# Early concatenation

In diesem Abschnitt werden die erzielten Ergebnisse und das Early-Concatenation Modell weitergehend untersucht, um folgenden Forschungsfragen zu beantworten:
- Wie ist die performance auf IEMOCAP und MELD
- Gibt es Änderungen in der Klassifizierung im Vergleich zum normalen InstructERC
  - Wenn ja, welche?
- Was wird durch die Akustik erkannt?
  - Was wird nur durch Akustik erkannt?
  - Gibt es Verbesserungen in bestimmten Emotionen
- Kann das Modell das volle Potenzial aus beiden Modalitäten ausnutzen?
- Nutzt das Modell die neuen Feature?
- Hat das Vortraining einen Einfluss auf die Effektivität des Modells

## IEMOCAP



In [16]:
DATASET = "IEMOCAP"


def get_result_dataframe(dataset: str) -> pd.DataFrame:
    normal_path = os.path.join(BASE_PATH, dataset, "preds_test_normal.json")
    audio_path = os.path.join(BASE_PATH, dataset, "preds_test_audio_only.json")

    normal_df = utils.build_result_dataframe(normal_path)
    audio_df = utils.build_result_dataframe(audio_path)
    ierc_df = utils.get_instructerc_results(INSTRUCTERC_BASE_PATH, dataset)

    results = utils.merge_result_dataframes([normal_df, audio_df, ierc_df], ["normal", "audio", "ierc"])
    results = utils.extract_dialogue_information(results)

    return results


df_iemocap = get_result_dataframe(DATASET)

len(df_iemocap)

314

In [18]:
df_iemocap[["input_x", "input_y"]].iloc[0]

input_x    Now you are expert of sentiment and emotional ...
input_y    Now you are expert of sentiment and emotional ...
Name: 0, dtype: object

### Performance

Dafür wird der gewichtete F1 Score berechnet:
\begin{equation}
    F1 = 2\cdot\frac{precision\cdot recall}{precision + recall}
\end{equation}
Dieser wird für jedes Label berechnet und durch die Anzahl der label normalisiert.
Dadurch bekommt jedes Label, unabhängig von der Anzah, die selbe Gewichtung im Gesamtergebnis.

In [9]:
from sklearn.metrics import f1_score


labels = df_iemocap["target"].value_counts().index.to_list()
f1_score(df_iemocap["target"], df_iemocap["output"], average="weighted", labels=labels)

0.6993065614123333

### Einfluss der Akustik

In [15]:
# TODO Merge mit instructerc, wenn der Index stimmt

audio_but_not_both_correct = df_iemocap[(df_iemocap["target"] == df_iemocap["output_audio"]) & (df_iemocap["target"] != df_iemocap["output"])]
print(f"Das Modell hat {len(audio_but_not_both_correct)} Beispiele nicht richtig erkannt, obwohl die Akustik das richtige Ergebnis geliefert hätte")

Das Modell hat 160 Beispiele nicht richtig erkannt, obwohl die Akustik das richtige Ergebnis geliefert hätte


## Attention-Weights

Um zu untersuchen, ob das training effektiv und das Modell die zusätzlichen feature tatsächlich verwendet, untersuchen wir die Attention-Weight Matrix.
Diese gibt aufschluss darüber, worauf sich das Modell konzentriert.
Zur erinnerung, die Attention-Matrix ist eine $T_x \times T_x$ Matrix, wobei $T_x$ für die Länge der Eingabesequenz steht.
Diese ist besitzt nur im unteren linken Dreieck $\{A_{i,j} \mid i \ge j\}$ Werte, während alle anderen 0 betragen.
Eine Begründung dafür steht in Abschnitt (ref...).
Jede Zeile enthält das Attenntion-Query Ergebnis eines Query-Vektors, während die Spalten die Ergebnisse der Keys representiert.
Zu interpretieren ist also eine Zeile, dass sich aus der neue Kontext-Vektor für den Token zu prozentualen Anteilen zusammensetzt, wie diese in der Zeile stehen.
Währenddessen zeigt jede Zeile, wie stark ein gegebener Token zum nächsten Zustand beigetragen hat.
Befinden sich in einer Spalte nur niedrige Werte, hat dieser Token wenig zum Ergebnisausgang beigetragen.

Wenn man also überprüfen möchte, ob das Netz die akustischen Feature tatsächlich verwendet, sollte es keine besonderen Auffälligkeiten oder erhöhte Werte in den Spalten der Audio-Token geben.

In [9]:
LANGUAGE_MODEL = os.path.abspath("../../models/language/LLaMA2-base")
LORA_ADAPTER = os.path.abspath("../../models/language/adapter/iemocap/LLaMA2-base")
ACOUSTIC_MODEL = os.path.abspath(
    "../../models/acoustic/wav2vec2/wav2vec2-large-robust-12-ft-emotion-msp-dim"
)
DS_TRAIN_PATH = os.path.abspath("../../datasets/iemocap/iemocap.csv")
DS_DEV_PATH = os.path.abspath("../../datasets/iemocap/iemocap.csv")
DS_TEST_PATH = os.path.abspath("../../datasets/iemocap/iemocap.csv")
STAGE1_PATH = os.path.abspath(
    "../../experiments/multimodal/concat/iemocap/LLaMA2-base/mlp/audio_instruction/stage_1"
)
STAGE2_PATH = os.path.abspath(
    "../../experiments/multimodal/concat/iemocap/LLaMA2-base/mlp/audio_instruction/stage_1"
)

In [12]:
def get_attention_weights(
    model: MmLlamaConcat,
    dataset_path: str,
    processor: MmLlamaProcessor,
    layer_idx=0,
    sample_index=0,
):
    sample = utils.get_sample(
        dataset_path,
        processor,
        [sample_index],
        collator_type=SequenceGenerationCollator,
        dataset_kwargs={"task": "normal", "audio_placement": "target"},
    )

    llama = model.llama
    att1 = llama.get_submodule(f"model.layers.{layer_idx}.self_attn")

    attention_weights = None

    def attention_hook(module, input, output):
        nonlocal attention_weights
        attention_weights = output[1]

    att_handle = att1.register_forward_hook(attention_hook)

    def prepate_nested_batch(batch: dict[dict[torch.Tensor]]):
        device = "cpu"
        if torch.cuda.is_available():
            device = "cuda"

        text = {k: v.to(device) for k, v in batch["text"].items()}
        acoustic = {k: v.half().to(device) for k, v in batch["acoustic"].items()}

        return {**text, **acoustic}

    with torch.no_grad():
        _ = model(**prepate_nested_batch(sample))

    att_handle.remove()
    return attention_weights[0].cpu().numpy()


def print_attention_weight_matrix(
    attention_weights: np.ndarray,
    sample,
    config: MmLlamaConfig,
    tokenizer: AutoTokenizer,
):
    head_norm = attention_weights.mean(axis=0)
    head_norm = np.apply_along_axis(lambda x: x / np.max(x), 1, head_norm)

    audio_token_id = config.audio_token_id

    token_ids = sample["text"]["input_ids"][0].cpu().numpy()
    audio_loc = np.where(token_ids == audio_token_id)[0][0]
    token_ids = np.concatenate(
        (token_ids[:audio_loc], [audio_token_id] * 10, token_ids[audio_loc:])
    )
    token_strings = np.array(tokenizer.convert_ids_to_tokens(token_ids))
    # token_strings[token_ids == 0] = ""

    mean_attention = np.mean(head_norm, axis=0, where=head_norm != 0)
    # print(mean_attention)
    top_token_ids_idx = np.argsort(mean_attention)[::-1][:25]
    # print(top_token_ids_idx)
    # last_token_ids_idx = np.argsort(mean_attention)[:10]
    audio_loc_idx = np.where(token_ids == audio_token_id)[0]
    selected_tokens = np.unique(np.concatenate([top_token_ids_idx, audio_loc_idx]))
    token_strings[~np.isin(np.arange(len(token_strings)), selected_tokens)] = ""
    # print(token_strings)

    plt.figure(figsize=(10, 10))
    plt.imshow(head_norm, cmap="viridis")
    plt.xticks(range(len(token_strings)), token_strings, rotation=90, fontsize=6)
    plt.yticks(range(len(token_strings)), token_strings, fontsize=6)
    plt.xlabel("Query Tokens")
    plt.ylabel("Key Tokens")
    plt.show()


In [11]:
def print_model_attention_weights(dataset_path: str, layer_idx=0, sample_index=0):
    model, config, processor = utils.get_model(
        LANGUAGE_MODEL, LORA_ADAPTER, ACOUSTIC_MODEL, STAGE1_PATH, MmLlamaConcat, {"output_attention_weights":True}
    )
    attention_weights = get_attention_weights(
        model, dataset_path, layer_idx, sample_index
    )
    print_attention_weight_matrix(
        attention_weights, sample_index, config, processor.tokenizer
    )


print_model_attention_weights(layer_idx=0, sample_index=0)

Um herauszufinden, welchen Einfluss das Vortraining auf die Verteilung der GEwichte hat, ist hier die Matrix eines Modells, welches direkt auf Stage 3 trainiert wurde.